# Building Fast Queries on a CSV

## The Dataset

In this project, we will be looking at data from an online laptop store, in order to answer a few different business questions about their inventory.  The data in the `laptops.csv` file which we will use was adapted from the [Laptop Prices dataset on Kaggle](https://www.kaggle.com/ionaskel/laptop-prices).

Each row of the dataset contains the following information about each laptop:

- **ID**: A unique identifier for the laptop
- **Company**: The name of the company that produces the laptop
- **Product**: The name of the laptop
- **TypeName**: The type of laptop
- **Inches**: The size of the screen in inches
- **ScreenResolution**: The resolution of the screen
- **CPU**: The laptop CPU
- **RAM**: The amount of RAM in the laptop
- **Memory**: The size of the hard drive
- **GPU**: The graphics card name
- **OpSys**: The name of the operating system
- **Weight**: The weight of the laptop
- **Price**: The price of the laptop

The purpose of the project is to create different methods to run these queries and find out which methods are the most efficient.

First, we will explore the data and print the first few rows.

In [1]:
import csv

with open('laptops.csv') as file:
    reader = csv.reader(file)
    laptops = list(reader)
    header = laptops[0]
    rows = laptops[1:]
    
print(header)

for row in rows[0:5]:
    print(row)
    print('\n')

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339']


['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898']


['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575']


['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537']


['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256

## Inventory Class

The goal of this project is to create a class that represents the inventory of the laptop store.  The methods in that class will implement queries that we want to answer about the inventory.  We will preprocess the data to make those queries run faster.

Some of the queries we want to answer are:

- Given a laptop id, find the corresponding data.
- Given an amount of money, find whether there are two laptops whose total price is that given amount.
- Identify all laptops whose price falls within a given budget.

We will start by implementing the constructor.

In [2]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            data = list(reader)
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
                
inventory = Inventory('laptops.csv')
print(inventory.header)
print(len(inventory.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303


## Finding a Laptop from the ID

We will be updating the `Inventory` class throughout the project in order to answer the queries listed previously.

The first thing we want to implement is a way to look up a laptop from a given identifier, so that, when a customer comes to the store with a purchase slip, they can quickly identify the laptop to which it corresponds.

To do this, we will write a function named `get_laptop_from_id()`.  This function will take the identifier of the laptop as an argument and return the full row of the laptop with that ID.

In [3]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            data = list(reader)
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
        
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id('3362737'))
print(inventory.get_laptop_from_id('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## Improving ID Lookups

The algorithm written above has to look at every single row to find the one specified or decide that it doesn't exist, giving it the time complexity *O(R)*, where *R* is the number of rows.

If the dataset was very large, this could slow down the application considerably, so it would be worthwhile for us to preprocess the data to make the algorithm quicker.  Sets and dictionaries can check in constant time whether a given identifier exists, so we could use one of these for our data, instead of a list of lists.  As we want to retrieve all the row information for the laptop, we will use a dictionary, which allows us to associate values to keys.

In the following code, we will edit the `__init__` method to preprocess the data into a dictionary where the keys are the laptop IDs and the values are the rows, then use that dictionary in the `get_laptop_from_id()` method.

In [4]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            data = list(reader)
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
            self.id_to_row = {}
            for row in self.rows:
                self.id_to_row[row[0]] = row
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None 
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id_fast('3362737'))
print(inventory.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


# Comparing the Performance

We have created two methods with which to find a laptop given an ID, namely `get_laptop_from_id()` and `get_laptop_from_id_fast()`.  The first one has the time complexity *O(R)*, where *R* is the number of rows, and the new method has the time complexity *O(1)* as it uses more memory to store the `self.id_to_row` dictionary and uses slightly more time creating the dictionary as an instance. Now, we will compare the performance of those two methods by generating random IDs and using both methods to lookup the same IDs.  We will use the `time` module to measure the execution time of each lookup, for each method, and add all the times together.

In [5]:
import time
import random

ids = [str(random.randint(1000000, 9999999)) for _ in range (10000)]

inventory = Inventory('laptops.csv')

### calculating time for lookup using `get_laptop_from_id()` ###
total_time_no_dict = 0
for id in ids:
    start = time.time()
    inventory.get_laptop_from_id(id)
    end = time.time()
    total_time_no_dict += end - start
    
### calculating time for lookup using `get_laptop_from_id_fast()` ###
total_time_dict = 0
for id in ids:
    start = time.time()
    inventory.get_laptop_from_id_fast(id)
    end = time.time()
    total_time_dict += end - start
    
print('Time with no dictionary:', total_time_no_dict)
print('Time with dictionary:', total_time_dict)

Time with no dictionary: 1.2291452884674072
Time with dictionary: 0.006325960159301758


Looking at the above times, we can see that using a dictionary makes the algorithm approximately (1.229/0.006) = **204** times faster than with no dictionary.

# Two Laptop Promotion

The online laptop store offers a gift card which can be used to buy up to two laptops.  We want to run a query to find out whether they can spend the amount of the card in full by buying either one laptop which costs the total amount on the card, or by buying two laptops whose prices add up to the total amount on the card.  We will update the `Inventory()` class and create this function in the following code.

In [6]:
### first we update the `Inventory()` class ###

class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            data = list(reader)
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
            self.id_to_row = {}
            for row in self.rows:
                self.id_to_row[row[0]] = row
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None 
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for i in self.rows:
            for j in self.rows:
                if i + j == dollars:
                    return True
        return False  
        
### then test the new function ###

inventory = Inventory('laptops.csv')

print(inventory.check_promotion_dollars(1000))
print(inventory.check_promotion_dollars(442))

True
False


# Optimising Laptop Promotion

As we did with the previous query of finding a laptop from an ID, we can also optimise the algorithm for `check_promotion_dollars()` by preprocessing the data.  As we only want to find out whether there is a True or False answer, we can use a set instead of a dictionary this time to store all of the laptop prices.  Then, we will be able to check in constant time whether there is a laptop with a given price within that set.

We will again update the `Inventory()` class below and add a new method.

In [7]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            data = list(reader)
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
            self.id_to_row = {}
            for row in self.rows:
                self.id_to_row[row[0]] = row
            self.prices = set()
            for row in self.rows:
                self.prices.add(row[-1])
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None 
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for i in self.rows:
            for j in self.rows:
                if i + j == dollars:
                    return True
        return False  
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for price in self.prices:
            if dollars - price in self.prices:
                return True
        return False   

Now we can test the `Inventory()` class using the new method.

In [8]:
inventory = Inventory('laptops.csv')

print(inventory.check_promotion_dollars_fast(1000))
print(inventory.check_promotion_dollars_fast(442))

True
False


# Comparing Promotion Functions

Now we will compare the performance of `check_promotion_dollars()` and `check_promotion_dollars_fast()` as we did before with the IDs function.

In [9]:
prices = [random.randint(100, 5000) for _ in range (100)]

inventory = Inventory('laptops.csv')

### testing the speed of `check_promotion_dollars()` ###
total_time_no_set = 0
for price in prices:
    start = time.time()
    inventory.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set += end - start
    
### testing the speed of `check_promotion_dollars_fast()` ###
total_time_set = 0
for price in prices:
    start = time.time()
    inventory.check_promotion_dollars_fast(price)
    end = time.time()
    total_time_set += end - start
    
print('Time with no set:', total_time_no_set)
print('Time with set:', total_time_set)

Time with no set: 57.642812728881836
Time with set: 0.0004494190216064453


We can calculate that the algorithm which uses preprocessing is (57.642813/0.000449) = approximately **13,000** times faster than without preprocessing.

# Finding Laptops Within a Budget

The final query we want to look at is to identify all laptops whose price falls within a certain budget.  

If we sort all laptops by price, we can then use binary search to identify the first laptop in the sorted list which has a price higher than the given amount.  This will make the search more efficient as it will stop as soon as it finds the first laptop which is over budget, rather than having to search the entire data set.

First, we will again update the `Inventory()` class to include a method which finds the first laptop which is more expensive than the given budget amount.

In [10]:
def row_price(row):
    return row[-1]

class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            data = list(reader)
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
            self.id_to_row = {}
            for row in self.rows:
                self.id_to_row[row[0]] = row
            self.prices = set()
            for row in self.rows:
                self.prices.add(row[-1])
            for row in self.rows:
                self.rows_by_price = sorted(self.rows, key = row_price)
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None 
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for i in self.rows:
            for j in self.rows:
                if i + j == dollars:
                    return True
        return False  
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for price in self.prices:
            if dollars - price in self.prices:
                return True
        return False   
    
    def find_first_laptop_more_expensive(self, target_price):
        range_start = 0
        range_end = len(self.rows_by_price) - 1
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2
            price = self.rows_by_price[range_middle][-1]
            if price == target_price:
                return range_middle
            elif price < target_price:
                range_start = range_middle + 1
            else:
                range_end = range_middle - 1
        price = self.rows_by_price[range_start][-1]
        if price != target_price:
            return -1
        return range_start

Now we will test the new function in the `Inventory()` class.

In [11]:
inventory = Inventory('laptops.csv')

print(inventory.find_first_laptop_more_expensive(1000))
print(inventory.find_first_laptop_more_expensive(10000))

682
-1


# Conclusion

In this project, we have looked at a dataset from an online laptop store containg information on all of the laptops in their inventory.

We have explored the use of different methods to answer questions about the data, with the intention of showing that algorithms can be made much more efficient by preprocessing the data. 

*It should be noted that, whenever we have tested a function using random values created by `random.randint()`, they will have slightly different results each time the code is run, but they give us an approximate idea of the picture.*